In [ ]:
using MLJ
using CSV
using DataFrames
using StatsPlots
using Random

In [ ]:
data = CSV.read("CASP.csv");
@show schema(data).scitypes
@show schema(data)
first(data, 4)

In [ ]:
coerce!(data, Count=>Continuous);

In [ ]:
describe(data)

In [ ]:
histogram(data["RMSD"], bins = :scott )

In [ ]:
@df data corrplot(cols(1:4))

In [ ]:
@df data corrplot(cols(5:8))

In [ ]:
y,X= unpack(data,==(:RMSD),colname -> true);

In [ ]:
models() do model
    matching(model, X, y) 
end

In [ ]:
train, test = partition(eachindex(y), 0.8,shuffle=true,rng=42)

In [ ]:
#RandomForestRegressor, ElasticNetCVRegressor, BayesianRidgeRegressor, SVMRegressor, DecisionTreeRegressor, AdaBoostRegressor
measures()

In [ ]:
Random.seed!(1234);
model_rf=@load RandomForestRegressor() pkg="DecisionTree"
model_rf=machine(model_rf,X,y)

eval_rf=evaluate!(model_rf, rows=train, resampling=CV(nfolds=10,shuffle=true,rng=42), measures=[mae,rms])
eval_rf.measurement

In [ ]:
Random.seed!(1234);
model_elnet=@load ElasticNetCVRegressor() 
model_elnet=machine(model_elnet,X,y)

eval_elnet=evaluate!(model_elnet, rows=train, resampling=CV(nfolds=10,shuffle=true,rng=42), measures=[mae,rms])
eval_elnet.measurement

In [ ]:
Random.seed!(1234);
model_svr=@load SVMRegressor() 
model_svr=machine(model_svr,X,y)

eval_svr=evaluate!(model_svr, rows=train, resampling=CV(nfolds=10,shuffle=true,rng=42), measures=[mae,rms])
eval_svr.measurement

In [ ]:
Random.seed!(1234);
model_bayes=@load BayesianRidgeRegressor() 
model_bayes=machine(model_bayes,X,y)

eval_bayes=evaluate!(model_bayes, rows=train, resampling=CV(nfolds=10,shuffle=true,rng=42), measures=[mae,rms])
eval_bayes.measurement

In [ ]:
Random.seed!(1234);
model_adabo=@load AdaBoostRegressor() 
model_adabo=machine(model_adabo,X,y)

eval_boost=evaluate!(model_adabo, rows=train, resampling=CV(nfolds=10,shuffle=true,rng=42), measures=[mae,rms])
eval_boost.measurement

In [ ]:
bar([eval_rf.measurement[1],eval_boost.measurement[1],eval_bayes.measurement[1],eval_svr.measurement[1],eval_elnet.measurement[1]])

In [ ]:
#grid search for rf, elasticnet and bayes